In [ ]:
import os, subprocess, msprime, pyslim, tskit
import numpy as np 
import matplotlib.pyplot as plt

This notebook briefly introduces the powerful `recapitate` functionality of SLiM/msprime available in the `pyslim` package, as shown in Example 4 of this paper:

https://onlinelibrary.wiley.com/doi/full/10.1111/1755-0998.12968

Galloway, J., Messer, P. W., Haller, B. C., Kelleher, J., & Ralph, P. L. (2018). Tree-sequence recording in SLiM opens new horizons for forward-time simulation of whole genomes. Molecular Ecology Resources, (November 2018), 552–566. https://doi.org/10.1111/1755-0998.12968

> In many applications one wishes to execute a non-neutral forward simulation beginning with an equilibrium amount of extant neutral genetic diversity, and the simulation needed to generate that pre-existing diversity, typically called the model “burn-in”, can take quite a long time – often much longer than it takes to execute the non-neutral portion of the simulation. For a model with a long chromosome or large population size, this burn-in can be so long as to limit the practical scale of the simulations that can be conducted. One solution to this is a “hybrid” approach, in which a forward simulation is initialized with the result of a (much faster) coalescent simulation (similar to Bhaskar 2014).

Here is a SLiM script. It is:

 - simulating a tree sequence 
 - holding a whole population, size 100 000
 - creating exactly one v strongly selected mutation at a site halfway through the simulated sequence
 - simulating forward-in-time until the mutation sweeps to fixation, or until 100 000 generations have passed (whichever happens first).


In [ ]:
slim_script = '''
initialize() { 
    initializeTreeSeq(); 
    initializeMutationRate(0); 
    initializeMutationType("m2", 0.5, "f", 1.0); // fitness coeff = 1.0
    m2.convertToSubstitution = F; 
    initializeGenomicElementType("g1", m2, 1); 
    initializeGenomicElement(g1, 0, 1e6 - 1); 
    initializeRecombinationRate(3e-10);
} 

1 late() { 
    sim.addSubpop("p1", 100000);
}

100 late() { 
    sample(p1.genomes, 1).addNewDrawnMutation(m2, 5e5);
}

100:10000 late() { 
    mut = sim.mutationsOfType(m2); 
    if (mut.size() != 1) 
        stop(sim.generation + ": LOST");
    else if (sum(sim.mutationFrequencies(NULL, mut)) == 1.0)
    { 
        sim.treeSeqOutput("ex4_TS_decap.trees"); 
        sim.simulationFinished();
    } 
}
'''

slim_file = open("slim_script.slim", "w")
slim_file.writelines(slim_script)
slim_file.close()

In [ ]:
%%time
# slim_run = subprocess.check_call(["slim", "slim_script.slim"])
slim_run = os.system("slim slim_script.slim")

The command above is equivalent to running this in the terminal:
```
slim slim_script.slim 
```
After some time you should have a SLiM-generated tree-sequence file called `ex4_TS_decap.trees`.
We'll load this using the `load` function in pyslim, and then use the `recapitate` method:

In [ ]:
# Run the SLiM model and load the resulting .trees file 
ts = pyslim.load("ex4_TS_decap.trees")

In [ ]:
%%time

# Recapitate
recap = ts.recapitate(recombination_rate=3e-10, Ne=1e5, random_seed=131) 

What do we expect to see? The region surrounding the selected locus should be younger, and so have smaller tree heights.

Indeed, this is what we see:

In [ ]:
# Plot tree heights after recapitation.
plt.hlines(100000, xmin=breakpoints[0], xmax=breakpoints[-1], linestyle='dashed',color='gray')
# plt.vlines(500000, ymin=-150000, ymax=1200000, linestyle='dashed', color='gray')
breakpoints = list(recap.breakpoints())[:-1]
heights = []
for tree in recap.trees():
    root_node = tree.root
    root_time = recap.node(root_node).time
    heights.append(root_time)
plt.step(breakpoints, heights, where='post') 
plt.show()

## What have we saved?

Our simulation took about 1 min (SLiM) + 0.5 sec (msprime).

What if we'd only used SLiM instead?

> Simulating the neutral burn‐in period in SLiM instead, with neutral mutations occurring at a rate of μ =10−7, would take an estimated 114.7 hr.

```(⌐■_■) ヽ(°◇° )ノ ヽ(°◇° )ノ ヽ(°◇° )ノ ヽ(°◇° )ノ```